In [1]:
from Data import load_data
from Model.Evaluation import Delta_E
from scipy import spatial
import numpy as np
from Model import Evaluation
from  matplotlib.colors import rgb_to_hsv
from matplotlib.colors import hsv_to_rgb

## Load data

In [2]:
keys = ["Seen_pairings",
        "Unseen_Pairings",
        "Unseen_Reference",
        "Unseen_Modifier",
        "Fully_Unseen",
        "Overall"
        ]

In [3]:
path_training_data = "Data/WM18_train.pkl"
path_testing_data = "Data/WM18_test.pkl"

In [4]:
Data_training = load_data.load_obj(path_training_data)
Data_testing = load_data.load_obj(path_testing_data)
Rmu, Wordvector, Tmu = load_data.training_data(Data_training)
E_Rmu, E_Wordvector, E_Tmu = load_data.testing_data(Data_testing)

In [5]:
train_prediction_HSV = "Predictions/Pred_HSV_Train.pkl"
train_prediction_RGB = "Predictions/Pred_RGB_Train.pkl"

test_prediction_HSV = "Predictions/Pred_HSV_Test.pkl"
test_prediction_RGB = "Predictions/Pred_RGB_Test.pkl"

In [6]:
training_Pred_HSV = load_data.load_obj(train_prediction_HSV)
training_Pred_RGB = load_data.load_obj(train_prediction_RGB)

testing_Pred_HSV = load_data.load_obj(test_prediction_HSV)
testing_Pred_RGB = load_data.load_obj(test_prediction_RGB)

In [7]:
train_tuples = [i[:3] for i in Data_training]
test_tuples = [i[:3] for i in Data_testing]

In [9]:
train_Cosine_RGB = [1 - spatial.distance.cosine(training_Pred_RGB[i]-Rmu[i], Tmu[i]- Rmu[i]) for i in range(len(Rmu))]

train_Cosine_HSV = [1 - spatial.distance.cosine(training_Pred_HSV[i]-Rmu[i], Tmu[i]- Rmu[i]) for i in range(len(Rmu))]

In [12]:
train_DeltaE_RGB = [Delta_E(training_Pred_RGB[i]-Rmu[i], Tmu[i]-Rmu[i]) for i in range(len(Rmu))]

train_DeltaE_HSV = [Delta_E(training_Pred_HSV[i]-Rmu[i], Tmu[i]-Rmu[i]) for i in range(len(Rmu))]

## Generating data

In [13]:
def encoder(number):
    if number == 0:
        return [1, 0]
    else:
        return [0, 1]

In [14]:
OneHot_CosinSimilarity = []
OneHot_DeltaEDistance = []
for i in range(len(train_Cosine_RGB)):
    Cosin = np.argmax([train_Cosine_RGB[i], train_Cosine_HSV[i]]) #, Cosine_Both[i]])
    DeltaE = np.argmin([train_DeltaE_RGB[i], train_DeltaE_HSV[i]]) #, DeltaE_Both[i]])
    OneHot_CosinSimilarity.append(encoder(Cosin))
    OneHot_DeltaEDistance.append(encoder(DeltaE))

In [16]:
Training_Cosine = []
Training_DeltaE = []

for index in range(len(train_Cosine_RGB)):
    Training_Cosine.append(OneHot_CosinSimilarity[index])
    Training_DeltaE.append(OneHot_DeltaEDistance[index])

Training_Cosine = np.array(Training_Cosine)
Training_DeltaE = np.array(Training_Cosine)

## Train

In [17]:
from keras.layers import Input, Dense
from keras.models import Model
import keras
from keras import backend as K

Using TensorFlow backend.
C:\Users\emano\Anaconda3\envs\Color-modifiers\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\emano\Anaconda3\envs\Color-modifiers\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\emano\Anaconda3\envs\Color-modifiers\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\emano\An

In [18]:
input_Rmu = Input(shape=(3,), dtype='float32', name='input_Rmu')
input_WordVector = Input(shape=(300,), dtype='float32', name='input_WordVector')

x_fc1 = keras.layers.concatenate([input_Rmu, input_WordVector])
x_fc1 = Dense(32, activation='sigmoid')(x_fc1)
x_fc1 = Dense(16, activation='relu')(x_fc1)


out_put = Dense(2, activation='softmax')(x_fc1)

model = Model(inputs=[input_Rmu, input_WordVector], outputs=out_put)

In [19]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta())

In [20]:
def decoder(index, OH_pred):
    vector_space = np.argmax(OH_pred)
    if vector_space == 0:
        return testing_Pred_RGB[index]
    elif vector_space == 1:
        return testing_Pred_HSV[index]

In [21]:
# %%capture
n = 12

result = {}

for i in range(n):
    history = model.fit([Rmu, Wordvector], 
                       Training_DeltaE,
                       epochs= 50,
                       batch_size=32)
    Pred = model.predict([E_Rmu, E_Wordvector])
    Final_pred = [decoder(i, Pred[i]) for i in range(len(Pred))]
    result[i] = Evaluation.evaluation(train_tuples, test_tuples,
                                     Final_pred, E_Rmu, E_Tmu)
    

Epoch 1/50
313/313 [==============================] - 2s 7ms/step - loss: 1.1107
Epoch 2/50
313/313 [==============================] - 0s 62us/step - loss: 0.7322
Epoch 3/50
313/313 [==============================] - 0s 63us/step - loss: 0.6217
Epoch 4/50
313/313 [==============================] - 0s 63us/step - loss: 0.5905
Epoch 5/50
313/313 [==============================] - 0s 65us/step - loss: 0.5795
Epoch 6/50
313/313 [==============================] - 0s 67us/step - loss: 0.5722
Epoch 7/50
313/313 [==============================] - 0s 67us/step - loss: 0.5678
Epoch 8/50
313/313 [==============================] - 0s 67us/step - loss: 0.5620
Epoch 9/50
313/313 [==============================] - 0s 60us/step - loss: 0.5580
Epoch 10/50
313/313 [==============================] - 0s 63us/step - loss: 0.5513
Epoch 11/50
313/313 [==============================] - 0s 65us/step - loss: 0.5474
Epoch 12/50
313/313 [==============================] - 0s 65us/step - loss: 0.5418
Epoch 13/50
31

313/313 [==============================] - 0s 63us/step - loss: 0.3325
Epoch 50/50
313/313 [==============================] - 0s 71us/step - loss: 0.3361
Epoch 1/50
313/313 [==============================] - 0s 68us/step - loss: 0.3315
Epoch 2/50
313/313 [==============================] - 0s 71us/step - loss: 0.3333
Epoch 3/50
313/313 [==============================] - 0s 68us/step - loss: 0.3233
Epoch 4/50
313/313 [==============================] - 0s 76us/step - loss: 0.3354
Epoch 5/50
313/313 [==============================] - 0s 73us/step - loss: 0.3350
Epoch 6/50
313/313 [==============================] - 0s 70us/step - loss: 0.3320
Epoch 7/50
313/313 [==============================] - 0s 73us/step - loss: 0.3335
Epoch 8/50
313/313 [==============================] - 0s 71us/step - loss: 0.3277
Epoch 9/50
313/313 [==============================] - 0s 70us/step - loss: 0.3428
Epoch 10/50
313/313 [==============================] - 0s 71us/step - loss: 0.3305
Epoch 11/50
313/313 [====

313/313 [==============================] - 0s 71us/step - loss: 0.2814
Epoch 46/50
313/313 [==============================] - 0s 74us/step - loss: 0.2788
Epoch 47/50
313/313 [==============================] - 0s 70us/step - loss: 0.2845
Epoch 48/50
313/313 [==============================] - 0s 65us/step - loss: 0.2837
Epoch 49/50
313/313 [==============================] - 0s 71us/step - loss: 0.2811
Epoch 50/50
313/313 [==============================] - 0s 70us/step - loss: 0.2886
Epoch 1/50
313/313 [==============================] - 0s 67us/step - loss: 0.2795
Epoch 2/50
313/313 [==============================] - 0s 73us/step - loss: 0.2772
Epoch 3/50
313/313 [==============================] - 0s 70us/step - loss: 0.2818
Epoch 4/50
313/313 [==============================] - 0s 70us/step - loss: 0.2870
Epoch 5/50
313/313 [==============================] - 0s 70us/step - loss: 0.2853
Epoch 6/50
313/313 [==============================] - 0s 68us/step - loss: 0.2763
Epoch 7/50
313/313 [==

313/313 [==============================] - 0s 68us/step - loss: 0.2638
Epoch 45/50
313/313 [==============================] - 0s 70us/step - loss: 0.2726
Epoch 46/50
313/313 [==============================] - 0s 65us/step - loss: 0.2724
Epoch 47/50
313/313 [==============================] - 0s 65us/step - loss: 0.2753
Epoch 48/50
313/313 [==============================] - 0s 68us/step - loss: 0.2681
Epoch 49/50
313/313 [==============================] - 0s 67us/step - loss: 0.2763
Epoch 50/50
313/313 [==============================] - 0s 68us/step - loss: 0.2654
Epoch 1/50
313/313 [==============================] - 0s 71us/step - loss: 0.2717
Epoch 2/50
313/313 [==============================] - 0s 67us/step - loss: 0.2645
Epoch 3/50
313/313 [==============================] - 0s 67us/step - loss: 0.2742
Epoch 4/50
313/313 [==============================] - 0s 71us/step - loss: 0.2647
Epoch 5/50
313/313 [==============================] - 0s 70us/step - loss: 0.2731
Epoch 6/50
313/313 [=

313/313 [==============================] - 0s 68us/step - loss: 0.2424
Epoch 41/50
313/313 [==============================] - 0s 68us/step - loss: 0.2431
Epoch 42/50
313/313 [==============================] - 0s 67us/step - loss: 0.2473
Epoch 43/50
313/313 [==============================] - 0s 73us/step - loss: 0.2483
Epoch 44/50
313/313 [==============================] - 0s 65us/step - loss: 0.2466
Epoch 45/50
313/313 [==============================] - 0s 65us/step - loss: 0.2476
Epoch 46/50
313/313 [==============================] - 0s 65us/step - loss: 0.2520
Epoch 47/50
313/313 [==============================] - 0s 73us/step - loss: 0.2407
Epoch 48/50
313/313 [==============================] - 0s 71us/step - loss: 0.2479
Epoch 49/50
313/313 [==============================] - 0s 68us/step - loss: 0.2601
Epoch 50/50
313/313 [==============================] - 0s 70us/step - loss: 0.2440


## Evaluation

In [23]:
Cosin, Delta = Evaluation.evaluation(train_tuples, test_tuples, Final_pred, E_Rmu, E_Tmu)

In [24]:
# Cosine Similarity
for k in keys:
    print(Cosin[k])

0.9551970518704846
0.7979094521585745
0.7995254646687572
0.6340031965819869
0.46196519191375357
0.8686575193579804


In [25]:
# Delta-E Distance
for k in keys:
    print(Delta[k])

3.5424675919994217
4.5454038470798555
7.704373391325542
10.40092532662398
13.721067544573247
5.381807523014021
